In [1]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm, trange
import tqdm
from pprint import pprint 
import statsmodels.api as sm
import math
import geemap
import os

from IPython.display import Image

import folium
from folium import plugins
import importlib

import geopandas as gpd
from PIL import Image

In [4]:
#ee.Authenticate()

Enter verification code: 4/1AX4XfWht0rDvLcz47WtrUHc9hHPzlZSIkNCpWUz5HBWkLy4m3kS6IsoiJjI

Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
col_sent = ee.ImageCollection("COPERNICUS/S2_SR")

In [7]:
# Isolate Cook County from the Census collection
counties = ee.FeatureCollection('TIGER/2018/Counties')
cookCounty = counties.filter(ee.Filter.eq("GEOID", '17031'))

# Get all US zip codes, then restrict to those that intersect with Cook County, and then
# filter out those that don't overlap at all but only touch on the boundary
# (e.g. some adjacent zip codes in Illinois and Indiana)
zipCodes = (
    ee.FeatureCollection('TIGER/2010/ZCTA5').filterBounds(cookCounty)
    .filter("GEOID10 != '46311'").filter("GEOID10 != '46324'")
    .filter("GEOID10 != '46321'").filter("GEOID10 != '46327'")
    .filter("GEOID10 != '46320'").filter("GEOID10 != '60047'")
    .filter("GEOID10 != '60035'").filter("GEOID10 != '60110'")
    .filter("GEOID10 != '60118'").filter("GEOID10 != '60143'")
    .filter("GEOID10 != '60191'").filter("GEOID10 != '60106'")
    .filter("GEOID10 != '60523'").filter("GEOID10 != '60441'")
    .filter("GEOID10 != '60448'").filter("GEOID10 != '60449'")
    .filter("GEOID10 != '60491'").filter("GEOID10 != '60417'")
)

# Create a geometry that is the union of all the zip codes in Chicago
zipChiGeom = zipCodes.union(100);
#censusTracts = ee.FeatureCollection("TIGER/2010/Tracts_DP1").filterBounds(cookCounty)

zipCodeIDs = zipCodes.aggregate_array("GEOID10").getInfo()
pprint(zipCodeIDs, width=90, compact=True)

['60636', '60077', '60164', '60415', '60422', '60469', '60526', '60601', '60602', '60603',
 '60604', '60606', '60607', '60612', '60618', '60619', '60621', '60631', '60638', '60639',
 '60641', '60645', '60646', '60651', '60652', '60654', '60655', '60656', '60661', '60706',
 '60707', '60712', '60714', '60805', '60026', '60029', '60043', '60053', '60076', '60104',
 '60130', '60131', '60141', '60153', '60154', '60155', '60160', '60162', '60163', '60165',
 '60171', '60176', '60195', '60203', '60301', '60302', '60304', '60425', '60426', '60428',
 '60455', '60456', '60457', '60459', '60647', '60471', '60659', '60558', '60070', '60482',
 '60487', '60107', '60609', '60634', '60629', '60093', '60804', '60133', '60018', '60527',
 '60464', '60521', '60173', '60453', '60461', '60624', '60091', '60626', '60169', '60062',
 '60015', '60472', '60827', '60643', '60467', '60411', '60622', '60439', '60476', '60126',
 '60202', '60458', '60465', '60637', '60803', '60501', '60478', '60201', '60610', '60423',

In [8]:
# Subset images with less than 5% cloud cover, then subset those which intersect the 
# lower left corner of the rectangle 'poly' we've just defined above.
clouds = col_sent.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)).filterBounds(cookCounty)

# Adjust chosen bands to anything you want, and the featurization will flow from it
chosen_bands = ['B2', 'B3', 'B4', 'B8']

# choose only desired bands from the image 
bands = clouds.select(chosen_bands)
s = bands.sort('CLOUD_COVER')
# s contains all sentinel images which satisfied the above two filters 

# Option 2:
scene = s.reduce(ee.Reducer.median())

In [9]:
# Applying the same filters as above, but only to the Near Infared Band (NIR):
nir = clouds.select(['B8']).reduce(ee.Reducer.median())

# Do the same for the red band
red = clouds.select(['B4']).reduce(ee.Reducer.median())

# Calculate NDVI (Normalized Difference Vegetation Index) and restrict it to the polygon that is made from
# the union of all Chicago zip codes
numer = nir.add(red)
denom = nir.subtract(red)
ndvi = denom.divide(numer)
ndvi_Chi = ndvi.clip(zipChiGeom.geometry())

In [10]:
surfTemp = ee.ImageCollection('MODIS/061/MOD11A1').filterDate('2018-01-01').filterBounds(zipChiGeom)
landSurfaceTemperature = surfTemp.select('LST_Day_1km');
landSurfaceTemperatureVis = {
  'min': 13000.0,
  'max': 16500.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]}

In [11]:
# Generate a map, add the ndvi as a layer, then center the map on Cook County, and add Chicago's zip codes
# as a layer
ndviVP = {'min': -1, 'max': 1, 'palette': ['000FFF', 'FFFFFF', '00FF00']}

fmap = geemap.Map()
fmap.addLayer(ndvi_Chi, ndviVP, 'Scene 1')
fmap.centerObject(cookCounty, 9)
#fmap.addLayer(cookCounty)
fmap.addLayer(
    landSurfaceTemperature, landSurfaceTemperatureVis,
    'Land Surface Temperature');
#tempChart = ui.Chart.image.regions(landSurfaceTemperature, zipCodes, ee.Reducer.mean(), scale, 'system:zipCodes', 'system:index')
#lst_urban_point = lst.mean().sample(cookCounty, scale).first().get('LST_Day_1km').getInfo()
fmap.addLayer(zipCodes)
#fmap.addLayer()
#fmap
#tempChart

In [14]:
landSurfaceTemperature
#print(landSurfaceTemperature.first().propertyNames())
print(landSurfaceTemperature.aggregate_array('LST_Day_1km'))

ee.List({
  "functionInvocationValue": {
    "functionName": "AggregateFeatureCollection.array",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_0_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Image.select",
                    "arguments": {
                      "bandSelectors": {
                        "constantValue": [
                          "LST_Day_1km"
                        ]
                      },
                      "input": {
                        "argumentReference": "_MAPPING_VAR_0_0"
                      }
                    }
                  }
                }
              }
            },
            "collection": {
              "f

In [16]:
# RUNNING THIS WILL DOWNLOAD A CSV CONTAINING MEAN NDVI BY ZIP CODE IN COOK COUNTY
# TO YOUR PERSONAL DOWNLOADS FOLDER. YOU CAN ALTERNATIVELY JUST COPY THE ndvi_chiZip_dataset.csv THAT IS
# IN MY FOLDER TO YOUR OWN ;).
    
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_stats = os.path.join(out_dir, 'chi_ST_stats_2018_01_01.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
#geemap.zonal_statistics(ndvi_Chi, zipCodes, out_stats, statistics_type='MEAN', scale=1000)
geemap.zonal_statistics(landSurfaceTemperature, zipCodes, out_stats, statistics_type='MEAN', scale=1000)

ndviVP = {'min': -1, 'max': 1, 'palette': ['000FFF', 'FFFFFF', '00FF00']}

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/matheuboucher/Downloads/chi_ST_stats_2018_01_01.csv
